In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import copy
import itertools
import threading, time


config_de = {"base": "https://de.wikipedia.org",
             "start_page": "https://de.wikipedia.org/wiki/Ernst-Th%C3%A4lmann-Park",
             "goal_page": "https://de.wikipedia.org/wiki/Unterhaltung_(Bauwesen)",
             "bad": ["/w/", "/Wikipedia", "/wiki/Hilfe", "/wiki/Spezial", "/wiki/Datei", "/wiki/Kategorie", 
                     "/wiki/Portal", "/wiki/Wikipedia:", "/wiki/Diskussion:"]}

config_en = {"base": "https://en.wikipedia.org",
             "start_page": "https://en.wikipedia.org/wiki/New_Horizons",
             "goal_page": "https://en.wikipedia.org/wiki/Silent_film",
             "bad": ["/w/", "/Wikipedia", "/wiki/Help", "/wiki/Special", "/wiki/File", "/wiki/Category", 
                     "/wiki/Portal", "/wiki/Wikipedia:", "/wiki/Discussion:"]}


# choose your config here
config = config_de

base = config["base"]
bad = config["bad"]

In [ ]:
def format_page(page):
    '''formats the page so it can be scraped. uses requests and BeautifulSoup libraries'''
    page_ready = BeautifulSoup(requests.get(page).text, "html.parser").body
    return page_ready


def prepare_bad_urls(base, bad):
    '''returns list of wikipedia subsites which shouldn't be used for the game'''
    full_bad_urls = []
    
    for bad_url in bad:
        full_bad_url = urllib.parse.urljoin(base, bad_url)
        full_bad_urls.append(full_bad_url)
    
    return full_bad_urls


def clean(link):
    full_url = urllib.parse.urljoin(base, link['href'])
        
    if "#" in full_url:
        full_url = full_url[:full_url.find("#")]
        
    if full_url == base:
        return "bad"
        
    if not full_url.startswith(base):
        return "bad"

        
    full_bad_urls = prepare_bad_urls(base, bad)

    is_bad_url = False
        
    for fbu in full_bad_urls:
        if full_url.startswith(fbu):
            is_bad_url = True
            return "bad"
                
        if is_bad_url == True:
            return "bad"
    
    return full_url


def return_wiki_links(page):
    '''returns wiki links contained in a page. needs format_page and prepare_bad_urls functions.'''
    page_text = format_page(page)
    links_in_page = page_text.find_all('a', href=True)
    
    wiki_links_in_page = list(set([clean(link) for link in links_in_page]))
    
    return list(set([link for link in wiki_links_in_page if link != "bad"]))

In [ ]:
class StoppableThread(threading.Thread):
    '''necessary edit of threading.Thread: progress thread needs to be 
    stopped once the goal page has been found'''
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._stop_event = threading.Event()
    
    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

class ProgressMeter(StoppableThread):
    '''implements status info (number of links searched) that stops once the goal page is found.
    .stop() must be called at the moment the goal page is found'''
    def run(self):
        time.sleep(30)
        total_secs = 30
        
        while not self.stopped():
            print("Stay tuned - {} links searched, none matched the goal page so far...".format(len(already_tried)))
            time.sleep(30)
            total_secs += 30
        
        print("In total, {} links were searched in {} minutes.".format(len(already_tried), round(total_secs / 60, 1)))

In [ ]:
def wikipedia_game(start_page, goal_page):
        
    if goal_page == start_page:
        print("DONE: no need for searching a connection, the two pages are identical.")
        return print("You are already on {}".format(goal_page))

    round_count = 1
    
    progressthread = ProgressMeter()
    progressthread.start()
    
    links_to_test = return_wiki_links(start_page)
    print("LET'S GO: Starting to search the shortest connection between {} and {}".format(start_page, goal_page))

    while True:
                
        if round_count > 1:
            links_to_test = copy.deepcopy(new_links)
        else:
            new_links = []
        
        if goal_page in links_to_test:
            return print("DONE: The goal page {} can be reached with one click: {} is a link in {}.".format(goal_page, goal_page, goal_page))
        else:
            print("Round {} completed without results. Starting round {}.".format(round_count, (round_count + 1)))
            round_count += 1

        for link in links_to_test: #creates list with all links relevant for the round.
            new_link_bunch = return_wiki_links(link)
            new_links_bunch = [x for x in new_link_bunch if x not in already_tried]
            
            if goal_page in new_links_bunch:
                progressthread.stop()
                return print("DONE: The goal page can be reached with {} clicks: {} connects {} with {}.".format(str(round_count), link, start_page, goal_page, len(already_tried + new_links_bunch)))
            
            new_links.append(new_links_bunch)
            [already_tried.append(link) for link in new_links_bunch]

        new_links = list(set(list(itertools.chain.from_iterable(new_links))))

In [ ]:
# now play the game:
already_tried = [config["start_page"]]

wikipedia_game(config["start_page"], config["goal_page"])